In [3]:
import numpy as np
import os
from ECG.eager_ops import expected_gradients as eg
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
path = "./eval_data_10k/"

In [5]:
os.listdir(path)

['cnn_ph',
 'cnn_ph_eg',
 'cnn_pl',
 'cnn_pl_eg',
 'lstm_ph',
 'lstm_ph_eg',
 'lstm_pl',
 'lstm_pl_eg',
 'teset_x.npy',
 'teset_y.npy']

In [70]:
# Creating the confusion matrices for PH
cnn_ph_cv1 = np.load(path+ph_folders[0]+"/cms_cv1.npy")

In [4]:
def get_cmss(foldername, tname):
    if tname == "ph":
        cmss = []
        for i in range(1,11):
            cmss.append(np.load(path+foldername+"/cms_cv"+str(i)+".npy").reshape(1,8,8))
        cmss = np.concatenate(tuple(cmss), axis=0)
        return cmss
    elif tname == "pl":
        return np.load(path+foldername+"/cms.npy")

def get_results(foldername, tname):
    if tname == "ph":
        results = []
        for i in range(1,11):
            results.append(np.load(path+foldername+"/results_cv"+str(i)+".npy", allow_pickle=True)[1:6])
        return np.stack(tuple(results))
    elif tname == "pl":
        return np.load(path+foldername+"/results.npy", allow_pickle=True)


In [5]:
def make_cm(cmss, name):
    categories=['N','L','R','V','A','F','f','/']
    plt.figure(figsize=(10,7))
    sns.heatmap(cmss.mean(axis=0), annot=True, xticklabels=categories, yticklabels=categories)
    plt.title(name)
    plt.show()

### Beat Holdout

In [6]:
ph_folders = ['cnn_ph','cnn_ph_eg','lstm_ph']

In [79]:
# a = get_cmss("cnn_ph", "ph")
# b = get_cmss("lstm_ph", "ph")
# c = get_cmss('cnn_ph_eg', "ph")
# d = get_cmss('cnn_ph_eg', "ph")
# make_cm(a, "CNN - Beat holdout Confusion Matrix")
# make_cm(b, "LSTM - Beat holdout Confusion Matrix")
# make_cm(c, "CNN - Beat holdout with ECG Confusion Matrix")
# make_cm(d, "LSTM - Beat holdout with ECG Confusion Matrix")

### Patient Leave-out

In [ ]:
pl_folders = ['cnn_pl','cnn_pl_eg','lstm_pl','lstm_pl_eg']

In [110]:
# e = get_cmss("cnn_pl", "pl")
# f = get_cmss("lstm_pl", "pl")
# g = get_cmss('cnn_pl_eg', "pl")
# h = get_cmss('lstm_pl_eg', "pl")

# make_cm(e, "CNN - Patient leaveout Confusion Matrix")
# make_cm(g, "CNN - Patient leaveout with ECG Confusion Matrix")
# make_cm(f, "LSTM - Patient leaveout Confusion Matrix")
# make_cm(h, "LSTM - Patient leaveout with ECG Confusion Matrix")

In [7]:
d_r = get_results("cnn_pl", "pl")
e_r = get_results("lstm_pl", "pl")
f_r = get_results('cnn_pl_eg', "pl")
g_r = get_results('lstm_pl_eg', "pl")
print(d_r[3][1:6])
print(f_r[0][1:6])
print(e_r[0][1:6])
print(g_r[0][1:6])

[0.828 0.413 0.484 0.434 0.828]
[0.81 0.446 0.472 0.428 0.81]
[0.631 0.381 0.337 0.294 0.631]
[0.655 0.386 0.309 0.279 0.655]


In [6]:
# Calculating attribution loss
def eg_att(foldername, tname):
    test_y = np.load("./eval_data_10k/teset_y.npy", allow_pickle=True)
    test_x = np.load("./eval_data_10k/teset_x.npy", allow_pickle=True)
    if tname=="ph":
        models = []
        attributions = []
        for i in range(1,11):
            m = tf.keras.models.load_model(path+foldername+"/Model_cv"+str(i)+".h5")
            att = eg(inputs=tf.convert_to_tensor(test_x[i-1].reshape(-1, test_x[i-1].shape[1], 1).astype(np.float32)), labels=tf.keras.utils.to_categorical(test_y[i-1].astype(np.float32)), model=m)
            print(i, test_x[i-1].shape, test_y[i-1].shape, att.shape)
            break
    # elif tname=="pl":

eg_att("cnn_ph_eg", "ph")


In [ ]:
# #cv1
m_cv1 = tf.keras.models.load_model(path+"cnn_ph_eg"+"/Model_cv1.h5")

NameError: name 'tf' is not defined

In [ ]:
test_y = np.load("./eval_data_10k/teset_y.npy", allow_pickle=True)
test_x = np.load("./eval_data_10k/teset_x.npy", allow_pickle=True)

In [ ]:
att = eg(inputs=test_x[0].reshape(-1, test_x[0].shape[1], 1).astype(np.float32), labels=tf.keras.utils.to_categorical(test_y[0].astype(np.float32)), model=m_cv1)

In [ ]:
tf.keras.utils.to_categorical(test_y[0]).shape

(10888, 9)

In [ ]:
test_x.reshape(-1, test_x.shape[1], 1).astype(np.float32)

IndexError: tuple index out of range